In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm  = ChatOpenAI(model_name="gpt-4",
                temperature=0.1,
                streaming=True,
                )

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

template = """ 
    You are a helpful AI taking to a human.
    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True, # 프롬프트 로그들을 확인할 수 있음
)

chain.predict(question="My name is nico.")



> Entering new LLMChain chain...
Prompt after formatting:
 
    You are a helpful AI taking to a human.
    
    Human:My name is nico.
    You:


> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [6]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
 
    You are a helpful AI taking to a human.
    Human: My name is nico.
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great, Nico! Seoul is a beautiful city. How can I assist you further?
Human: What is my name?
AI: Your name is Nico.
    Human:I live in Seoul
    You:


> Finished chain.


'Yes, you mentioned that you live in Seoul, Nico. How may I assist you further?'

In [7]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
 
    You are a helpful AI taking to a human.
    Human: My name is nico.
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great, Nico! Seoul is a beautiful city. How can I assist you further?
Human: What is my name?
AI: Your name is Nico.
Human: I live in Seoul
AI: Yes, you mentioned that you live in Seoul, Nico. How may I assist you further?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Nico.'